In [94]:
using CSV
using DataFrames
using Random

include("../src/data.jl");
Random.seed!(104);

### Creación del dataset

In [95]:
if !isfile("../resources/dataset.csv");
    dataset_folder = "../resources/csv_files";
    dataset = CreateDataset(dataset_folder);
    CSV.write("../resources/dataset.csv", dataset);
else
    dataset = loadDataset("dataset", "../resources"); 
end;

### Summary

In [96]:
n_ins, n_vars = size(dataset);
n_inv = length(unique(dataset[:, 1]));
println("Número de variables: ", n_vars - 2);
println("Número de instancias: ", n_ins);
println("Número de individuos: ", n_inv);
println("Número de clases: ", length(unique(dataset[:, end])));

Número de variables: 561
Número de instancias: 10299
Número de individuos: 30
Número de clases: 6


### Gestión de ausentes

In [97]:
df_cols = eachcol(dataset);
missing_indx = findall.(ismissing, df_cols);
uncomplete_cols = .!isempty.(missing_indx);
missing_ratios = (length.(missing_indx) ./ n_ins) * 100;
println.("col: ", names(dataset[:, uncomplete_cols]), "| ratio de nulos: ", missing_ratios[uncomplete_cols], "%");
println("Ratio sobre el total: ", mean(missing_ratios),"%");

col: tBodyAcc-std()-X| ratio de nulos: 0.01941936110301971%
col: tBodyAcc-std()-Z| ratio de nulos: 0.01941936110301971%
col: tBodyAcc-mad()-X| ratio de nulos: 0.01941936110301971%
col: tBodyAcc-mad()-Y| ratio de nulos: 0.029129041654529564%
col: tBodyAcc-mad()-Z| ratio de nulos: 0.029129041654529564%
col: tBodyAcc-max()-X| ratio de nulos: 0.01941936110301971%
col: tBodyAcc-max()-Z| ratio de nulos: 0.009709680551509855%
col: tBodyAcc-sma()| ratio de nulos: 0.009709680551509855%
col: tBodyAcc-iqr()-X| ratio de nulos: 0.029129041654529564%
col: tBodyAcc-iqr()-Y| ratio de nulos: 0.009709680551509855%
col: tBodyAcc-entropy()-Z| ratio de nulos: 9.991261287503642%
col: tBodyAcc-arCoeff()-Z,2| ratio de nulos: 9.991261287503642%
col: tBodyAcc-correlation()-X,Y| ratio de nulos: 9.991261287503642%
col: tGravityAcc-std()-Y| ratio de nulos: 9.991261287503642%
col: tGravityAcc-std()-Z| ratio de nulos: 9.991261287503642%
col: tGravityAcc-mad()-Z| ratio de nulos: 9.991261287503642%
col: tGravityAcc-ma

In [98]:
for c in names(dataset)
    if eltype(dataset[!, c]) <: Union{Missing, Float64}
        μ = mean(skipmissing(dataset[!, c]))
        dataset[!, c] = coalesce.(dataset[!, c], μ)
        # Force column type to Float64
        dataset[!, c] = Float64.(dataset[!, c])
    end
end

println("Valores ausentes sustituidos con la media de cada columna")
println("Número de ausentes: ", sum(count.(ismissing, eachcol(dataset))))

Valores ausentes sustituidos con la media de cada columna
Número de ausentes: 0


### HoldOut

In [99]:
train_index, test_index = holdOut(n_inv, 0.10);
test_dataset = dataset[findall(x -> any(x .== test_index), dataset[:, "subject"]), :];
train_dataset = dataset[findall(x -> any(x .== train_index), dataset[:, "subject"]), :];
println("Individuos en el dataset de test: ", test_index);
println("Individuos en el dataset de train: ", train_index);
println("Dimensiones de test: ", size(test_dataset), " Dimensiones de train: ", size(train_dataset));

Individuos en el dataset de test: [8, 24, 26]
Individuos en el dataset de train: [25, 14, 30, 13, 15, 3, 6, 7, 10, 19, 17, 27, 20, 16, 5, 12, 28, 4, 29, 18, 21, 2, 23, 11, 9, 1, 22]
Dimensiones de test: (1054, 563) Dimensiones de train: (9245, 563)


### OneHotEncoding

In [100]:
train_targets = OneHotEncoding(train_dataset[!, "Activity"]);
train_inputs = train_dataset[!, Not("Activity")];
test_targets = OneHotEncoding(test_dataset[!, "Activity"]);
test_inputs = test_dataset[!, Not("Activity")];
println(size(train_targets), size(train_inputs));
println(size(test_targets), size(test_inputs));

(9245, 6)(9245, 562)
(1054, 6)(1054, 562)


### CrossValidation

In [101]:
cross_val_index = crossvalidation(train_inputs[!, "subject"], 5);
cv_info = [(length(findall(x -> x == fold, cross_val_index)), unique(train_inputs[findall(x -> x == fold, cross_val_index), "subject"])) for fold in 1:5];
println("Fold | Size  | Individuals");
println("-----+-------+---------------------------");
for (i, (size, inds)) in enumerate(cv_info);
    inds_str = join(inds, ", ");
    print(i, "\t", size, "\t", inds_str, "\n");
end;
train_inputs = train_inputs[!, Not("subject")]
test_inputs = test_inputs[!, Not("subject")]

Fold | Size  | Individuals
-----+-------+---------------------------
1	2084	5, 16, 17, 22, 29, 30
2	2060	4, 6, 15, 19, 20, 27
3	1679	3, 7, 10, 18, 23
4	1584	1, 2, 9, 12, 13
5	1838	11, 14, 21, 25, 28


Row,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,tBodyAcc-max()-Y,tBodyAcc-max()-Z,tBodyAcc-min()-X,tBodyAcc-min()-Y,tBodyAcc-min()-Z,tBodyAcc-sma(),tBodyAcc-energy()-X,tBodyAcc-energy()-Y,tBodyAcc-energy()-Z,tBodyAcc-iqr()-X,tBodyAcc-iqr()-Y,tBodyAcc-iqr()-Z,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,tBodyAcc-entropy()-Z,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,2","tBodyAcc-arCoeff()-X,3","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1","tBodyAcc-arCoeff()-Y,2","tBodyAcc-arCoeff()-Y,3","tBodyAcc-arCoeff()-Y,4","tBodyAcc-arCoeff()-Z,1","tBodyAcc-arCoeff()-Z,2","tBodyAcc-arCoeff()-Z,3","tBodyAcc-arCoeff()-Z,4","tBodyAcc-correlation()-X,Y","tBodyAcc-correlation()-X,Z","tBodyAcc-correlation()-Y,Z",tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAcc-std()-X,tGravityAcc-std()-Y,tGravityAcc-std()-Z,tGravityAcc-mad()-X,tGravityAcc-mad()-Y,tGravityAcc-mad()-Z,tGravityAcc-max()-X,tGravityAcc-max()-Y,tGravityAcc-max()-Z,tGravityAcc-min()-X,tGravityAcc-min()-Y,tGravityAcc-min()-Z,tGravityAcc-sma(),tGravityAcc-energy()-X,tGravityAcc-energy()-Y,tGravityAcc-energy()-Z,tGravityAcc-iqr()-X,tGravityAcc-iqr()-Y,tGravityAcc-iqr()-Z,tGravityAcc-entropy()-X,tGravityAcc-entropy()-Y,tGravityAcc-entropy()-Z,"tGravityAcc-arCoeff()-X,1","tGravityAcc-arCoeff()-X,2","tGravityAcc-arCoeff()-X,3","tGravityAcc-arCoeff()-X,4","tGravityAcc-arCoeff()-Y,1","tGravityAcc-arCoeff()-Y,2","tGravityAcc-arCoeff()-Y,3","tGravityAcc-arCoeff()-Y,4","tGravityAcc-arCoeff()-Z,1","tGravityAcc-arCoeff()-Z,2","tGravityAcc-arCoeff()-Z,3","tGravityAcc-arCoeff()-Z,4","tGravityAcc-correlation()-X,Y","tGravityAcc-correlation()-X,Z","tGravityAcc-correlation()-Y,Z",tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,tBodyAccJerk-mean()-Z,tBodyAccJerk-std()-X,tBodyAccJerk-std()-Y,tBodyAccJerk-std()-Z,tBodyAccJerk-mad()-X,tBodyAccJerk-mad()-Y,tBodyAccJerk-mad()-Z,tBodyAccJerk-max()-X,tBodyAccJerk-max()-Y,tBodyAccJerk-max()-Z,tBodyAccJerk-min()-X,tBodyAccJerk-min()-Y,tBodyAccJerk-min()-Z,tBodyAccJerk-sma(),tBodyAccJerk-energy()-X,tBodyAccJerk-energy()-Y,tBodyAccJerk-energy()-Z,tBodyAccJerk-iqr()-X,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.277354,-0.0108438,-0.113844,-0.998925,-0.976382,-0.998735,-0.999021,-0.975701,-0.998473,-0.943903,-0.549295,-0.829735,0.850777,0.691864,0.843673,-0.993471,-0.999995,-0.999732,-0.999944,-0.998308,-0.978077,-0.998736,-0.777403,-0.339294,-0.806073,0.386823,-0.0895924,0.00252939,0.291626,0.252813,-0.224074,0.166204,0.0508126,0.251313,-0.213376,0.416205,-0.394083,-0.12207,0.0553176,0.420145,0.973298,-0.0884095,0.0801924,-0.998259,-0.953972,-0.993398,-0.9983,-0.982888,-0.993885,0.898758,-0.108667,0.0727104,0.992274,-0.0638001,0.0807462,-0.592776,0.925706,-0.99044,-0.989001,-0.998294,-0.981956,-0.995059,-1.0,-1.0,-0.766807,-0.342856,0.387653,-0.433224,0.479629,-0.434277,0.382189,-0.365987,0.366171,-0.387069,0.408418,-0.429856,0.448842,0.807129,-0.96367,-0.828516,0.0760575,0.0101041,-0.00252408,-0.996865,-0.984125,-1.0,-0.996929,-0.981924,-1.0,-0.99484,-0.992049,-0.997786,0.995023,0.987882,0.99826,-0.99602,-0.999974,-0.999722,-1.0,-0.996585,⋯
2,0.282293,-0.022769

### Normalización

In [102]:
train_cols = eachcol(train_inputs);
norm_values = minimum.(train_cols), maximum.(train_cols);
for fold in unique(cross_val_index)
    fold_indx = findall(x -> x == fold, cross_val_index)
    fold_train_cols = eachcol(train_inputs[fold_indx, :])
    fold_norm_values = minimum.(fold_train_cols), maximum.(fold_train_cols);    
    train_inputs[fold_indx, :] .-= fold_norm_values[1]';
    train_inputs ./= (fold_norm_values[2] .- fold_norm_values[1])'
end;
test_inputs .-= norm_values[1]';
test_inputs ./= (norm_values[2] .- norm_values[1])'

Row,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,tBodyAcc-max()-Y,tBodyAcc-max()-Z,tBodyAcc-min()-X,tBodyAcc-min()-Y,tBodyAcc-min()-Z,tBodyAcc-sma(),tBodyAcc-energy()-X,tBodyAcc-energy()-Y,tBodyAcc-energy()-Z,tBodyAcc-iqr()-X,tBodyAcc-iqr()-Y,tBodyAcc-iqr()-Z,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,tBodyAcc-entropy()-Z,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,2","tBodyAcc-arCoeff()-X,3","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1","tBodyAcc-arCoeff()-Y,2","tBodyAcc-arCoeff()-Y,3","tBodyAcc-arCoeff()-Y,4","tBodyAcc-arCoeff()-Z,1","tBodyAcc-arCoeff()-Z,2","tBodyAcc-arCoeff()-Z,3","tBodyAcc-arCoeff()-Z,4","tBodyAcc-correlation()-X,Y","tBodyAcc-correlation()-X,Z","tBodyAcc-correlation()-Y,Z",tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAcc-std()-X,tGravityAcc-std()-Y,tGravityAcc-std()-Z,tGravityAcc-mad()-X,tGravityAcc-mad()-Y,tGravityAcc-mad()-Z,tGravityAcc-max()-X,tGravityAcc-max()-Y,tGravityAcc-max()-Z,tGravityAcc-min()-X,tGravityAcc-min()-Y,tGravityAcc-min()-Z,tGravityAcc-sma(),tGravityAcc-energy()-X,tGravityAcc-energy()-Y,tGravityAcc-energy()-Z,tGravityAcc-iqr()-X,tGravityAcc-iqr()-Y,tGravityAcc-iqr()-Z,tGravityAcc-entropy()-X,tGravityAcc-entropy()-Y,tGravityAcc-entropy()-Z,"tGravityAcc-arCoeff()-X,1","tGravityAcc-arCoeff()-X,2","tGravityAcc-arCoeff()-X,3","tGravityAcc-arCoeff()-X,4","tGravityAcc-arCoeff()-Y,1","tGravityAcc-arCoeff()-Y,2","tGravityAcc-arCoeff()-Y,3","tGravityAcc-arCoeff()-Y,4","tGravityAcc-arCoeff()-Z,1","tGravityAcc-arCoeff()-Z,2","tGravityAcc-arCoeff()-Z,3","tGravityAcc-arCoeff()-Z,4","tGravityAcc-correlation()-X,Y","tGravityAcc-correlation()-X,Z","tGravityAcc-correlation()-Y,Z",tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,tBodyAccJerk-mean()-Z,tBodyAccJerk-std()-X,tBodyAccJerk-std()-Y,tBodyAccJerk-std()-Z,tBodyAccJerk-mad()-X,tBodyAccJerk-mad()-Y,tBodyAccJerk-mad()-Z,tBodyAccJerk-max()-X,tBodyAccJerk-max()-Y,tBodyAccJerk-max()-Z,tBodyAccJerk-min()-X,tBodyAccJerk-min()-Y,tBodyAccJerk-min()-Z,tBodyAccJerk-sma(),tBodyAccJerk-energy()-X,tBodyAccJerk-energy()-Y,tBodyAccJerk-energy()-Z,tBodyAccJerk-iqr()-X,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.638677,0.494578,0.443078,0.000537385,0.0118092,0.00063229,0.000489435,0.0121069,0.000763255,0.0280486,0.250179,0.0851326,0.925388,0.845932,0.912144,0.00326441,2.33e-6,0.000133985,2.811e-5,0.000846125,0.0109613,0.000605616,0.111298,0.330353,0.0969637,0.693411,0.455204,0.512365,0.645813,0.626406,0.387963,0.588803,0.525406,0.625656,0.421551,0.708103,0.302958,0.438965,0.527659,0.710073,0.986649,0.455795,0.540096,0.000870565,0.0230141,0.00330117,0.000849845,0.00855625,0.00349589,0.949379,0.445666,0.536355,0.996137,0.4681,0.540373,0.203612,0.962853,0.00478006,0.00549937,0.000851057,0.00902198,0.00245515,0.0,0.0,0.116597,0.328572,0.693827,0.308113,0.739815,0.282862,0.691094,0.317007,0.683086,0.329398,0.67546,0.314326,0.694649,0.903565,0.0181652,0.085742,0.538029,0.505052,0.498738,0.00156734,0.00679166,-0.00012271,0.0015357,0.00758537,-0.000204502,0.00257988,0.00389599,0.00110687,0.997511,0.993941,0.99913,0.00199008,1.3145e-5,0.000126537,-1.88e-6,0.00166797,⋯
2,0

In [103]:
if !isfile("variables/train_dataset.jld2")
    @save "variables/train_dataset.jld2" train_inputs train_targets
    @save "variables/test_dataset.jld2" test_inputs test_targets
    @save "variables/cross_val_index.jdl2" cross_val_index
end;
